# EDA on Retail Sales Data

This notebook performs Exploratory Data Analysis (EDA) on retail sales data. Replace the file paths below with your Dataset 1 and Dataset 2 links or local CSVs.

In [ ]:
# Configure your data sources here
DATASET1_PATH = '../data/dataset1_retail_sales.csv'  # Replace with your Dataset 1 Link or local path
DATASET2_PATH = '../data/dataset2_product_catalog.csv'  # Replace with your Dataset 2 Link or local path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

# Matplotlib display settings (keep default colors as per instructions)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

## 1) Data Loading and Initial Peek

In [ ]:
# Load Dataset 1 (Retail Sales) and Dataset 2 (Product Catalog)
df_sales = pd.read_csv(DATASET1_PATH)
df_products = pd.read_csv(DATASET2_PATH)

print("Sales shape:", df_sales.shape)
print("Products shape:", df_products.shape)

display(df_sales.head())
display(df_products.head())

## 2) Data Cleaning

In [ ]:
# Parse dates and basic cleaning
df_sales['order_date'] = pd.to_datetime(df_sales['order_date'], errors='coerce')
before = len(df_sales)
df_sales = df_sales.drop_duplicates()
df_sales = df_sales.dropna(subset=['order_id','order_date','customer_id','product_id','quantity','unit_price','revenue'])
after = len(df_sales)
print(f"Removed {before - after} rows during basic cleaning.")

# Ensure correct dtypes
df_sales['quantity'] = pd.to_numeric(df_sales['quantity'], errors='coerce').astype(int)
df_sales['unit_price'] = pd.to_numeric(df_sales['unit_price'], errors='coerce').astype(float)
df_sales['revenue'] = pd.to_numeric(df_sales['revenue'], errors='coerce').astype(float)

display(df_sales.describe())

## 3) Descriptive Statistics

In [ ]:
numeric_cols = ['quantity','unit_price','revenue','discount_rate'] if 'discount_rate' in df_sales.columns else ['quantity','unit_price','revenue']
desc = df_sales[numeric_cols].describe().T
display(desc)

# Additional stats
for col in numeric_cols:
    mean = df_sales[col].mean()
    median = df_sales[col].median()
    std = df_sales[col].std()
    print(f"{col}: mean={mean:.2f}, median={median:.2f}, std={std:.2f}")

## 4) Time Series Analysis: Monthly Revenue Trend

In [ ]:
monthly = (df_sales
           .set_index('order_date')
           .resample('M')['revenue']
           .sum()
           .reset_index())
monthly['month'] = monthly['order_date'].dt.to_period('M').astype(str)

# Plot monthly revenue
plt.figure()
plt.plot(monthly['order_date'], monthly['revenue'])
plt.title('Monthly Revenue Trend')
plt.xlabel('Month')
plt.ylabel('Revenue')
plt.tight_layout()
plt.show()

display(monthly.tail())

## 5) Customer Analysis

In [ ]:
# Top customers by total revenue
cust_rev = df_sales.groupby('customer_id')['revenue'].sum().reset_index().sort_values('revenue', ascending=False)
top_customers = cust_rev.head(10)
display(top_customers)

plt.figure()
plt.bar(top_customers['customer_id'], top_customers['revenue'])
plt.title('Top 10 Customers by Revenue')
plt.xlabel('Customer ID')
plt.ylabel('Revenue')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Simple RFM-like metrics (Recency, Frequency, Monetary)
ref_date = df_sales['order_date'].max() + pd.Timedelta(days=1)
rfm = df_sales.groupby('customer_id').agg(
    recency=('order_date', lambda x: (ref_date - x.max()).days),
    frequency=('order_id', 'nunique'),
    monetary=('revenue', 'sum')
).reset_index()

display(rfm.sort_values(['monetary','frequency'], ascending=[False, False]).head(10))

## 6) Product & Category Analysis

In [ ]:
# Merge category if missing
if 'category' not in df_sales.columns and 'product_id' in df_sales.columns:
    df_sales = df_sales.merge(df_products[['product_id','category']], on='product_id', how='left')

cat_rev = df_sales.groupby('category')['revenue'].sum().reset_index().sort_values('revenue', ascending=False)
display(cat_rev)

plt.figure()
plt.bar(cat_rev['category'], cat_rev['revenue'])
plt.title('Revenue by Category')
plt.xlabel('Category')
plt.ylabel('Revenue')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

prod_rev = df_sales.groupby('product_id')['revenue'].sum().reset_index().sort_values('revenue', ascending=False).head(15)
display(prod_rev)

plt.figure()
plt.bar(prod_rev['product_id'], prod_rev['revenue'])
plt.title('Top 15 Products by Revenue')
plt.xlabel('Product ID')
plt.ylabel('Revenue')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 7) Heatmap: Category vs Month (Revenue)

In [ ]:
df_sales['year_month'] = df_sales['order_date'].dt.to_period('M').astype(str)
pivot = df_sales.pivot_table(index='category', columns='year_month', values='revenue', aggfunc='sum').fillna(0)

# Plot heatmap using matplotlib (no seaborn)
plt.figure()
plt.imshow(pivot.values, aspect='auto')
plt.title('Revenue Heatmap: Category vs Month')
plt.xlabel('Month')
plt.ylabel('Category')
plt.xticks(ticks=range(len(pivot.columns)), labels=pivot.columns, rotation=90)
plt.yticks(ticks=range(len(pivot.index)), labels=pivot.index)
plt.colorbar()
plt.tight_layout()
plt.show()

pivot.tail()

## 8) Recommendations

In [ ]:
# Simple, data-driven recommendation placeholders
recs = []

# Identify slow months
if 'monthly' in locals() and not monthly.empty:
    slow_month = monthly.sort_values('revenue').head(1).iloc[0]
    recs.append(f"Consider targeted promotions during the slowest month: {slow_month['month']}.")

# Identify best category
if not cat_rev.empty:
    top_cat = cat_rev.iloc[0]
    recs.append(f"Allocate more inventory and marketing to best-performing category: {top_cat['category']}.")

# High-value customers
if not rfm.empty:
    hv = rfm.sort_values('monetary', ascending=False).head(1).iloc[0]
    recs.append(f"Create loyalty offers for high-value customer segment similar to: {hv['customer_id']}.")

for i, r in enumerate(recs, 1):
    print(f"{i}. {r}")